In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastai.vision.all import *
from fastbook import *
from mnist import MNIST

In [72]:
def init_params(size, std=0.1):
    return (torch.randn(size)*std).requires_grad_()

def create_label(x):
    t = torch.zeros(10)
    t[x] = 1
    return t


mndata = MNIST('datasets/mnist/')
images, labels = mndata.load_training()
xd = Tensor(images).float()
yd = Tensor(labels).float()

weights = init_params((10, 28*28, 1)).float()
bias = init_params((10, 1)).float()
lr = 0.005
train_images = torch.tensor(images).float() / 255.0
train_labels = torch.stack([create_label(x) for x in labels]).unsqueeze(2)

dset = list(zip(train_images, train_labels))

In [3]:

#do poprawy???

def linear1(train_data, weights, bias):
    return (train_data@weights).T.squeeze(0).unsqueeze(2) + bias

def loss_sme(predictions, targets):
    return (targets - predictions.sigmoid()).pow(2).mean(dim=0).sqrt()
    #return (targets - predictions).abs().mean()

def calc_grad(xb, yb, model, weights, bias):
    preds = model(xb, weights, bias)
    loss = loss_sme(preds, yb)
    for l in loss:
        l.backward(retain_graph=True)

def train_epoch(model, dl, lr, params, bias):
    for xb, yb in dl:
        calc_grad(xb, yb, model, params, bias)
        for i in range(len(params)):
            weights[i].data -= weights.grad[i].data * lr
            weights.grad[i].zero_()
        for bs in bias:
            bias.data -= bias.grad.data * lr
            bias.grad.zero_()
    print(batch_accuracy(linear1(train_images, weights, bias), train_labels).item())
            
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    return (preds.max(dim=1)[1] == yb.max(dim=1)[1]).float().mean()
    

In [73]:
dset = list(zip(train_images, train_labels))

In [82]:
dl = DataLoader(dset, batch_size=64, shuffle=True)

In [83]:
for _ in range(90):
    train_epoch(linear1, dl, lr, weights, bias)

0.9139833450317383
0.9139166474342346
0.9141666889190674
0.9141333103179932
0.9140833616256714
0.9142166376113892
0.9142000079154968
0.9140999913215637
0.9140166640281677
0.9144999980926514
0.9143333435058594
0.9143000245094299
0.9146000146865845
0.9144166707992554
0.9146166443824768
0.9146166443824768
0.9144999980926514
0.9146333336830139
0.9146166443824768
0.9146666526794434
0.9143999814987183
0.9144333600997925
0.914816677570343
0.9146166443824768
0.9147666692733765
0.914900004863739
0.9147999882698059
0.9149333238601685
0.914900004863739
0.9150166511535645
0.914650022983551
0.9149500131607056
0.9149500131607056
0.9150166511535645
0.9152833223342896
0.9150833487510681
0.915149986743927
0.9154499769210815
0.9152833223342896
0.9154333472251892
0.9154333472251892
0.9153833389282227
0.9153000116348267
0.9153500199317932
0.9154833555221558
0.9154000282287598
0.9158499836921692
0.9156166911125183
0.9156500101089478
0.9158666729927063
0.9156833291053772
0.9156833291053772
0.915983319282531

In [87]:
test = mndata.load_testing()

In [98]:
test_images = torch.tensor(test[0]).float() / 255.0
test_labels = torch.stack([create_label(x) for x in test[1]]).unsqueeze(2)

In [100]:
test_images.shape

torch.Size([10000, 784])

In [101]:
train_images.shape

torch.Size([60000, 784])

In [102]:
predictions = linear1(test_images, weights, bias)

In [104]:
batch_accuracy(predictions, test_labels).item()

0.9193000197410583